# Packages

In [ ]:
!apt install ghostscript python3-tk
!pip install opencv-python
!pip install camelot-py
!pip install beautifultable
!pip install opencv-python
!pip install pypdf2
!pip install openpyxl
!pip install ghostscript
!pip install gradio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 49 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 

In [ ]:
!pip install ghostscript

In [ ]:
import camelot
from beautifultable import BeautifulTable
import os
import sys
import pandas as pd
from typing import List
import PyPDF2
import gradio as gr

#  Extracting the tabular data

In [ ]:
# Function to extract tables from PDF and convert them into DataFrames
def get_tables_from_all_pages(path: str):
    all_tables = []
    # Get the total number of pages in the PDF
    with open(path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        total_pages = len(reader.pages)

    # Iterate through all pages
    for page in range(1, total_pages + 1):  # Camelot uses 1-based page numbers
        table_list = camelot.read_pdf(path, pages=str(page))
        if table_list.n > 0:
            for tab in range(table_list.n):
                # Convert the table to a DataFrame
                table_df = table_list[tab].df

                # Clean up the table (renaming columns, removing newlines, etc.)
                table_df = (
                    table_df.rename(columns=table_df.iloc[0])
                    .drop(table_df.index[0])
                    .reset_index(drop=True)
                )
                table_df = table_df.apply(lambda x: x.str.replace('\n', ''))

                # Clean column names
                table_df.columns = [col.replace('\n', ' ').replace(' ', '') for col in table_df.columns]
                table_df.columns = [col.replace('(', '').replace(')', '') for col in table_df.columns]

                # Append the cleaned DataFrame to the list of all tables
                all_tables.append(table_df)  # Store the page number with the table
    return all_tables

In [ ]:
# Function to save DataFrames as Excel
def save_tables_to_excel(df_list, output_excel: str):
    # Create an ExcelWriter object to write multiple sheets
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        # Iterate over each DataFrame in the list and save it to a separate sheet
        for i, table_df in enumerate(df_list):
            sheet_name = f'Table_{i+1}'  # Generate unique sheet name for each table
            table_df.to_excel(writer, sheet_name=sheet_name, index=False)

    return output_excel  # Return the Excel file path

# Gradio UI

In [ ]:
# Gradio function to process PDF and generate Excel
def process_pdf(pdf_file):
    # Extract tables from the uploaded PDF file
    df_list = get_tables_from_all_pages(pdf_file.name)

    # Save the tables into an Excel file
    output_excel = 'camelot_output.xlsx'
    save_tables_to_excel(df_list, output_excel)

    return output_excel  # Return the path to the Excel file for download

# Gradio Interface
pdf_input = gr.File(label="Upload PDF File")
excel_output = gr.File(label="Download Excel File")

interface = gr.Interface(
    fn=process_pdf,
    inputs=pdf_input,
    outputs=excel_output,
    title="PDF Table Extractor",
    description="Upload a PDF and download extracted tables as an Excel file with each table in a separate sheet."
)

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae75c524be6eaa7c13.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
